In [42]:
import sys
import os

sys.path.append("../Pre_processing")

from Data_Preparation_Library import *
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [2]:
#selectedBatches=["6","7","8","9","10","11","12","13","14","15","16","17","18","19","20"]
maxBatchId = 1
selectedBatches=[str(i) for i in range(maxBatchId)]

batch_data_object = []
for i in selectedBatches:
    with open(os.path.join(temp_path,'full_data_object_' + i + '.p'), 'rb') as handle:
        batch_data_object+=pickle.load(handle)

data_train = batch_data_object[0:int(len(batch_data_object)*2/3)]
data_test  = batch_data_object[int(len(batch_data_object)*2/3):]

In [3]:
def flux(d):
  # TODO: get flux_i from model
    if d.i_image != None:
        flux_i = sum(sum(d.i_image))
    else:
        flux_i = 0
    flux_g = sum(sum(d.g_image))
    #return flux_i+flux_g, flux_g-flux_i
    return flux_g

In [4]:
with open(os.path.join(output_path,"encoder_results_train_v2_" + "_".join(selectedBatches)),'rb') as handle:
    encoded_imgs1=pickle.load(handle)
    encoded_imgs_reshaped_train = [e.reshape(-1,1) for e in encoded_imgs1]

with open(os.path.join(output_path,"encoder_results_test_v2_" + "_".join(selectedBatches)),'rb') as handle:
    encoded_imgs2=pickle.load(handle)
    encoded_imgs_reshaped_test = [e.reshape(-1,1) for e in encoded_imgs2]


In [78]:
def pca_feature(d):
    
#  return [v1_i,v2_i,v1_i/v2_i,v2_i/v1_i, v1_g,v2_g,v1_g/v2_g,v2_g/v1_g]


SyntaxError: unexpected EOF while parsing (<ipython-input-78-f89bb59a94fa>, line 3)

In [91]:
# accessible in the class: size of image, maximum of image, normalization factor of g + fitted value of normalization factor for i
# define a function to get the training and testing dataset

def obtain_train_data():
    train_features = []
    train_labels = []
    for i in range(len(data_train)):
        if data_train[i].logMstar!=-99:
            e = encoded_imgs_reshaped_train[i]
            d = data_train[i]
            #f1,f2 = flux(d)
            f = flux(d)
            size_1=d.g_image.shape[0]
            size_2=d.g_image.shape[1]
            img_max=d.g_image.max()
            train_features.append(np.append(e,np.array([f,size_1,size_2,img_max])))
            train_labels.append(d.logMstar)
    return train_features, train_labels

train_features = np.asarray(obtain_train_data()[0])
train_labels = np.asarray(obtain_train_data()[1])

print(train_features.shape, train_labels.shape)

(659, 4100) (659,)


In [92]:
def obtain_test_data():
    test_features = []
    test_labels = []
    for j in range(len(data_test)):
        if data_test[j].logMstar!=-99:
            e = encoded_imgs_reshaped_test[j]
            d = data_test[j]

        #f1,f2 = flux(d)
            f = flux(d)
            size_1=d.g_image.shape[0]
            size_2=d.g_image.shape[1]
            img_max=d.g_image.max()
            test_features.append(np.append(e,np.array([f,size_1,size_2,img_max])))
            test_labels.append(d.logMstar)

    return test_features, test_labels

test_features = np.asarray(obtain_test_data()[0])
test_labels = np.asarray(obtain_test_data()[1])

print(test_features.shape, test_labels.shape)

(325, 4100) (325,)


In [93]:
# ## transform features
# train_features = StandardScaler().fit_transform(train_features)
# test_features = StandardScaler().transform(test_features)
# #features = MinMaxScaler().fit_transform(features)

X_scaler = StandardScaler()
train_features = X_scaler.fit_transform(train_features)
test_features = X_scaler.transform(test_features)

print(train_features[:5], test_features[:5])

[[-0.46823421 -0.5080603   0.02156935 ..., -0.08512842 -0.08512842
  -0.31407447]
 [-0.37778186  0.01002213  0.36101057 ...,  1.97956789  1.97956789
  -0.3734218 ]
 [-0.31466022 -0.04653447  0.02059651 ...,  1.35231838  1.35231838
   1.8731427 ]
 [-0.34544549  0.51801735  0.59461292 ..., -0.45102397 -0.45102397
  -0.4427597 ]
 [ 0.5452832   1.31677986  0.74994534 ..., -0.56863325 -0.56863325
  -0.42390013]] [[ 0.15133955 -1.55998065 -0.05018915 ...,  0.01941317  0.01941317
  -0.41906929]
 [-0.31818065  0.05129635  0.23653529 ...,  1.888094    1.888094
  -0.11246488]
 [ 0.8351674   1.77883233  1.02854044 ..., -0.49022706 -0.49022706
  -0.44070063]
 [-0.48722797 -0.36853051  0.03476775 ...,  0.01941317  0.01941317
  -0.30787221]
 [-0.51023902 -1.52314199 -1.47030972 ..., -0.52943016 -0.52943016
  -0.42655866]]


In [94]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers.core import Dropout

In [146]:
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=train_features.shape[1]))
model.add(Dropout(0.1))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1,activation='relu'))
model.compile(optimizer='Adadelta',
              loss='mse',
              metrics=['mse'])

In [147]:
model.fit(train_features, train_labels, epochs=100, batch_size=32,shuffle=True,
                validation_data=(test_features, test_labels))

Train on 659 samples, validate on 325 samples
Epoch 1/100
659/659 [==============================] - 0s - loss: 49.4105 - mean_squared_error: 49.4105 - val_loss: 16.3509 - val_mean_squared_error: 16.3509
Epoch 2/100
659/659 [==============================] - 0s - loss: 16.6761 - mean_squared_error: 16.6761 - val_loss: 9.4215 - val_mean_squared_error: 9.4215
Epoch 3/100
659/659 [==============================] - 0s - loss: 9.4558 - mean_squared_error: 9.4558 - val_loss: 8.3818 - val_mean_squared_error: 8.3818
Epoch 4/100
659/659 [==============================] - 0s - loss: 7.4211 - mean_squared_error: 7.4211 - val_loss: 6.5050 - val_mean_squared_error: 6.5050
Epoch 5/100
659/659 [==============================] - 0s - loss: 5.7277 - mean_squared_error: 5.7277 - val_loss: 6.1297 - val_mean_squared_error: 6.1297
Epoch 6/100
659/659 [==============================] - 0s - loss: 5.1393 - mean_squared_error: 5.1393 - val_loss: 5.1976 - val_mean_squared_error: 5.1976
Epoch 7/100
659/659 [===

659/659 [==============================] - 0s - loss: 2.1665 - mean_squared_error: 2.1665 - val_loss: 2.7395 - val_mean_squared_error: 2.7395
Epoch 54/100
659/659 [==============================] - 0s - loss: 2.0714 - mean_squared_error: 2.0714 - val_loss: 2.4513 - val_mean_squared_error: 2.4513
Epoch 55/100
659/659 [==============================] - 0s - loss: 2.0632 - mean_squared_error: 2.0632 - val_loss: 2.8572 - val_mean_squared_error: 2.8572
Epoch 56/100
659/659 [==============================] - 0s - loss: 2.0304 - mean_squared_error: 2.0304 - val_loss: 2.5620 - val_mean_squared_error: 2.5620
Epoch 57/100
659/659 [==============================] - 0s - loss: 1.9261 - mean_squared_error: 1.9261 - val_loss: 2.2198 - val_mean_squared_error: 2.2198
Epoch 58/100
659/659 [==============================] - 0s - loss: 2.0851 - mean_squared_error: 2.0851 - val_loss: 2.5267 - val_mean_squared_error: 2.5267
Epoch 59/100
659/659 [==============================] - 0s - loss: 1.6717 - mean_sq

In [148]:
type(test_predict),type(test_labels[1])

(list, numpy.float64)

In [149]:
from sklearn.metrics import mean_squared_error

test_predict = []
test_labels_2=[]
for i in range(100):
    if data_test[i].logMstar!=-99:
        test_predict.append(model.predict(test_features[i].reshape(1,-1))[0][0])
        test_labels_2.append(test_labels[i])
print(len(test_predict))
print(np.sqrt(mean_squared_error(np.asarray(test_predict), np.asarray(test_labels_2))))


100
1.21271062831
